In [12]:
"""
Theoretical metric proven "good articles" of the wikispeedia dataset.

Osc, nov 2024
"""

'\nTheoretical metric proven "good articles" of the wikispeedia dataset.\n\nOsc, nov 2024\n'

In [13]:
#pip install -r requirements.txt

In [1]:
from src.load_data import *
from src.graph_data import *
import pandas as pd

['/Users/oscargoudet/Desktop/ada-2024-project-adaholiks', '/opt/anaconda3/envs/ada_project/lib/python312.zip', '/opt/anaconda3/envs/ada_project/lib/python3.12', '/opt/anaconda3/envs/ada_project/lib/python3.12/lib-dynload', '', '/opt/anaconda3/envs/ada_project/lib/python3.12/site-packages', '/Users/oscargoudet/Desktop']
---- WARNING: The following articles have a problem: ----
Badugi has no links (out-edges) listed.
Color_Graphics_Adapter has no links (out-edges) listed.
Directdebit has no category listed.
Directdebit has no links (out-edges) listed.
Donation has no category listed.
Donation has no links (out-edges) listed.
Douglas_DC-4 has no links (out-edges) listed.
Duchenne_muscular_dystrophy has no links (out-edges) listed.
Friend_Directdebit has no category listed.
Klinefelter%27s_syndrome has no links (out-edges) listed.
Local_community has no links (out-edges) listed.
Lone_Wolf_%28gamebooks%29 has no links (out-edges) listed.
Osteomalacia has no links (out-edges) listed.
Pikachu

### Checking if there are any

In [2]:
from collections import defaultdict

total_pairs = 0
unconnected_pairs = 0
connected_pairs = [] 

for source in G.nodes():

    for target in G.nodes():

        if source != target:

            total_pairs += 1

            if nx.has_path(G, source, target):

                connected_pairs.append((source, target))

            else:
                
                unconnected_pairs += 1

print(f"Total pairs of nodes: {total_pairs}")
print(f"Number of unconnected pairs: {unconnected_pairs}")


KeyboardInterrupt: 

### List of theoretical "good nodes"

Here, for every node, we count the times it is part of a shortest path. By doing this, we obtain a list of nodes that are often good choices when playing the wikispeedia game as they are more often than not part of a shortest path.

In [6]:
node_counts = defaultdict(int)

for source, target in connected_pairs:

    all_paths = list(nx.all_shortest_paths(G, source=source, target=target))

    for path in all_paths:

        for node in path:

            if node != source and node != target:

                node_counts[node] += 1


node_counts_df = pd.DataFrame(node_counts.items(), columns=['Node', 'ShortestPathCount'])

node_counts_df
